# Download necessary files

This notebook downloads files necessary to replicate the analysis in Depsky et al. 2023.

In [1]:
import sys

sys.path.append("..")

In [2]:
from pathlib import Path

import pandas as pd
import requests
from cartopy.io import shapereader
from pyCIAM.io import (
    download_and_extract_from_zenodo,
    download_and_extract_partial_zip,
    get_zenodo_file_list,
)
from pyCIAM.utils import copy
from shared import (
    DIR_SHP,
    DIR_SLR_AR5_IFILES_RAW,
    DIR_SLR_AR6_RAW,
    DIR_SLR_SWEET_RAW,
    LOCALIZESL_COREFILES,
    LOCALIZESL_REV,
    PATH_BORDERS,
    PATH_COASTLINES,
    PATH_DIAZ_INPUTS_RAW,
    PATH_GADM,
    PATH_MOVEFACTOR_DATA,
    PATH_PWT,
    PATH_SLIIDERS,
    PATH_SLIIDERS_INCOME_INTERMEDIATE_FILE,
    PATH_SLR_AR5_QUANTILES,
    PATH_SLR_GMSL_HIST_TIMESERIES,
    PATH_SLR_HIST_TREND_MAP,
    PATHS_SURGE_LOOKUP,
    save,
)

In [3]:
Z_URL_BASE = "https://zenodo.org/api/"
Z_URL_RECORDS = Z_URL_BASE + "records/{doi}"
Z_URL_DEPOSITS = Z_URL_BASE + "deposit/depositions/{doi}"

# This will need to point to the correct version of the SLIIDERS zenodo store (see
# Depsky et al. 2023 for the version associated with that manuscript)
Z_SLIIDERS_DOI = "7693868"
Z_PYCIAM_DOI = "8229860"
Z_AR6_DOI = "6382554"
Z_SWEET_DOI = "6067895"


DOWNLOAD_DIAZ_INPUTS = True
DOWNLOAD_SLIIDERS = True
DOWNLOAD_SURGE_LOOKUPS = True

DOWNLOAD_PLOTTING_DATA = True

DOWNLOAD_SLR_AR5 = True
DOWNLOAD_SLR_AR6 = True
DOWNLOAD_SLR_SWEET = True

# Only needed if you would like to re-run LocalizeSL to re-generate AR5 SLR inputs
# (requires matlab or octave). The output of this workflow, quantiled to the quantiles
# we are working with in Depsky et al. 2023, is obtained with DOWNLOAD_SLR_AR5=True
DOWNLOAD_LOCALIZESL_INPUTS = False

# pre-release
# PARAMS = {"access_token": environ["ACCESS_TOKEN"]}
# Z_URL_SLIIDERS_PC = Z_URL_DEPOSITS

# post-release
PARAMS = {}
Z_URL_SLIIDERS_PC = Z_URL_RECORDS

In [5]:
pyciam_files = get_zenodo_file_list(Z_PYCIAM_DOI)

## Data for plotting and results calculation

These are necessary to run the [pyCIAM-results-figures.ipynb](./pyCIAM-results-figures.ipynb) notebook, but are not directly used in the model execution.

### Natural Earth Country Boundaries

In [6]:
if DOWNLOAD_PLOTTING_DATA:
    DIR_SHP.mkdir(exist_ok=True, parents=True)
    # Natural Earth coastline data
    if not PATH_BORDERS.is_file():
        print("Downloading Natural Earth borders data...")
        fname_boundary = Path(
            shapereader.natural_earth(
                resolution="10m",
                category="cultural",
                name="admin_0_boundary_lines_land",
            )
        )
        copy(fname_boundary.parent, PATH_BORDERS.parent)

    if not PATH_COASTLINES.is_file():
        print("Downloading Natural Earth coastlines data...")
        fname_coastlines = Path(
            shapereader.natural_earth(
                resolution="10m", category="physical", name="coastline"
            )
        )
        copy(fname_coastlines.parent, PATH_COASTLINES.parent)

    # GADM
    print("Downloading GADM data...")
    if not PATH_GADM.is_file():
        GADM_NAME = PATH_GADM.stem
        download_and_extract_partial_zip(
            PATH_GADM.parent,
            f"https://geodata.ucdavis.edu/gadm/gadm4.1/{GADM_NAME}.zip",
            f"{GADM_NAME}.gpkg",
        )

    # Penn World Table
    print("Downloading PWT data...")
    if not PATH_PWT.is_file():
        save(
            pd.read_excel("https://www.rug.nl/ggdc/docs/pwt100.xlsx", sheet_name=2),
            PATH_PWT,
        )

    # SLIIDERS intermediate output used for normalizing costs by GDP for presentation in
    # Depsky et al. 2023
    print("Downloading SLIIDERS intermediate file...")
    download_and_extract_from_zenodo(
        PATH_SLIIDERS_INCOME_INTERMEDIATE_FILE, pyciam_files, "inputs/ypk_"
    )

    # Output of the non-market relocation cost analysis in Depsky et al. 2023
    print("Downloading movefactor analysis outputs...")
    download_and_extract_from_zenodo(
        PATH_MOVEFACTOR_DATA, pyciam_files, "products/suboptimal"
    )

## SLIIDERS

In [7]:
if DOWNLOAD_SLIIDERS:
    print("Downloading SLIIDERS...")
    sliiders_files = requests.get(
        Z_URL_SLIIDERS_PC.format(doi=Z_SLIIDERS_DOI), params=PARAMS
    ).json()["files"]
    download_and_extract_from_zenodo(
        PATH_SLIIDERS, sliiders_files, "products/sliiders-v"
    )

## Diaz 2016 inputs

In [8]:
if DOWNLOAD_DIAZ_INPUTS:
    print("Downloading Diaz 2016 inputs...")
    download_and_extract_from_zenodo(
        PATH_DIAZ_INPUTS_RAW, pyciam_files, "inputs/diaz2016_inputs"
    )

## SLR Inputs

In [48]:
if DOWNLOAD_SLR_AR5:
    print("Downloading LocalizeSL pre-computed quantiles...")
    download_and_extract_from_zenodo(
        PATH_SLR_AR5_QUANTILES, pyciam_files, "inputs/ar5-msl"
    )

if DOWNLOAD_SLR_SWEET:
    print("Downloading SLR projections from Sweet 2022...")
    sweet_files = requests.get(
        Z_URL_RECORDS.format(doi=Z_SWEET_DOI), params=PARAMS
    ).json()["files"]
    download_and_extract_from_zenodo(
        DIR_SLR_SWEET_RAW,
        sweet_files,
        "Interagency_Report.zip",
        zip_glob=[
            "Results/TR_global_projections.nc",
            "Results/TR_gridded_projections.nc",
            "Results/TR_local_projections.nc",
        ],
    )

if DOWNLOAD_SLR_AR6:
    ar6_files = requests.get(Z_URL_RECORDS.format(doi=Z_AR6_DOI), params=PARAMS).json()[
        "files"
    ]

    # get total SLR
    for scope, name in [("global", "ar6"), ("regional", "ar6-regional-confidence")]:
        print(f"Downloading AR6 SLR projections: total, {scope}...")
        download_and_extract_from_zenodo(
            DIR_SLR_AR6_RAW / scope,
            ar6_files,
            f"{name}.zip",
            zip_glob=(
                f"{name}/{scope}/confidence_output_files/**/ssp*/total_*values.nc"
            ),
        )

    # get only the contribution of vertical land motion
    print("Downloading AR6 SLR projections: verticallandmotion, regional...")
    download_and_extract_from_zenodo(
        DIR_SLR_AR6_RAW / "regional",
        ar6_files,
        "ar6-regional-confidence.zip",
        zip_glob=(
            "ar6-regional-confidence/regional/confidence_output_files/**/ssp*/"
            "verticallandmotion_*values.nc"
        ),
    )

if DOWNLOAD_SLR_AR5 or DOWNLOAD_SLR_SWEET:
    # must also add some historical trends to convert from 2000 to 2005 MSL datum
    BASE_URL = "https://data.aviso.altimetry.fr/aviso-gateway/data/indicators/msl/"
    print("Downloading map of historical SLR...")
    with PATH_SLR_HIST_TREND_MAP.open("wb") as f:
        f.write(
            requests.get(
                BASE_URL + "MSL_Map_MERGED_Global_AVISO_NoGIA_Adjust.nc"
            ).content
        )
    print("Downloading historical GMSL timeseries...")
    with PATH_SLR_GMSL_HIST_TIMESERIES.open("wb") as f:
        f.write(
            requests.get(
                BASE_URL + "MSL_Serie_MERGED_Global_AVISO_GIA_Adjust_Filter2m.nc"
            ).content
        )

https://zenodo.org/api/files/d1eda86e-cb3d-4cda-9e12-32f9f35b9197/Interagency_Report.zip
/tmp/ciam/data/raw/slr/sweet2022/TR_global_projections.nc
Results/TR_global_projections.nc
/tmp/ciam/data/raw/slr/sweet2022/TR_gridded_projections.nc
Results/TR_gridded_projections.nc
/tmp/ciam/data/raw/slr/sweet2022/TR_local_projections.nc
Results/TR_local_projections.nc
https://zenodo.org/api/files/6cbb54eb-b2aa-483e-8dd0-179103857395/ar6.zip
/tmp/ciam/data/raw/slr/ar6/global/total_ssp126_low_confidence_values.nc
ar6/global/confidence_output_files/low_confidence/ssp126/total_ssp126_low_confidence_values.nc
/tmp/ciam/data/raw/slr/ar6/global/total_ssp245_low_confidence_values.nc
ar6/global/confidence_output_files/low_confidence/ssp245/total_ssp245_low_confidence_values.nc
/tmp/ciam/data/raw/slr/ar6/global/total_ssp585_low_confidence_values.nc
ar6/global/confidence_output_files/low_confidence/ssp585/total_ssp585_low_confidence_values.nc
/tmp/ciam/data/raw/slr/ar6/global/total_ssp119_medium_confidenc

## Storm Surge Lookup Tables

In [50]:
if DOWNLOAD_SURGE_LOOKUPS:
    print("Downloading segment-level storm surge lookup...")
    download_and_extract_from_zenodo(
        PATHS_SURGE_LOOKUP["seg"], pyciam_files, "inputs/surge-lookup-v1.1-seg."
    )

    print("Downloading segment/admin unit-level storm surge lookup...")
    download_and_extract_from_zenodo(
        PATHS_SURGE_LOOKUP["seg_adm"], pyciam_files, "inputs/surge-lookup-v1.1-seg_adm"
    )

## LocalizeSL Inputs

In [54]:
if DOWNLOAD_LOCALIZESL_INPUTS:
    DIR_SLR_AR5_IFILES_RAW.mkdir(exist_ok=True, parents=True)

    for corefile_name in LOCALIZESL_COREFILES:
        print(f"Downloading LocalizeSL corefile: {corefile_name}...")
        url = (
            f"https://github.com/bobkopp/LocalizeSL/raw/{LOCALIZESL_REV}/IFILES/"
            f"{corefile_name}.mat"
        )

        filename = Path(url).name
        with (DIR_SLR_AR5_IFILES_RAW / filename).open("wb") as f:
            f.write(requests.get(url).content)